In [1]:
import os
import shutil

import tensorflow as tf
#print(tf.__version__)
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [3]:
####https://www.tensorflow.org/tutorials/text/classify_text_with_bert

In [4]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=125)
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import  precision_score
from sklearn.metrics import  f1_score

In [6]:
import matplotlib.pyplot as plt
 
import seaborn as sns
sns.set()

In [7]:
bert_run_precision = []
bert_run_recall = []
bert_run_f1score = []
bert_run_accuracy = []

In [8]:
bert_model_name = 'bert_en_cased_L-12_H-768_A-12' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/2',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3


In [9]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [10]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [32]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='comment')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [33]:
classifier_model_test = build_classifier_model()

classifier_model_test.summary()
#bert_raw_result = classifier_model_test(tf.constant(text_test))
#print(tf.sigmoid(bert_raw_result))

Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
comment (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_word_ids': ( 0           comment[0][0]                    
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'sequence_output':  108310273   preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
______________________________________________________________________________________

In [34]:
#loading data

import pandas as pd
data = pd.read_excel('politeness_2k_data.xlsx')

In [35]:
data.fillna(value='', inplace=True)

In [36]:
data=data.drop(['final_agreed_rating'], axis=1)

In [37]:
data=data.drop(['stanford_tool_score'], axis=1)

In [38]:
initial_train=data.fillna("fillna").values


In [39]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [41]:
import numpy as np
fold_no = 1
import pandas as pd
from sklearn.model_selection import train_test_split
for train_index, test_index in KFold(10).split(initial_train):
    print("Starting Fold: ", fold_no)
    
    x_train, x_val = initial_train[train_index], initial_train[test_index]
    #print(initial_train)
    
    train = pd.DataFrame(data=x_train, index=[i for i in range(x_train.shape[0])], columns=[ "comment", "target"])
     
    val = pd.DataFrame(data=x_val, index=[i for i in range(x_val.shape[0])], columns=["comment", "target"])
    
    x_train, x_test= train_test_split(x_train,  test_size=0.11115, random_state=125)

    train = pd.DataFrame(data=x_train, index=[i for i in range(x_train.shape[0])], columns=["comment", "target"])
    test = pd.DataFrame(data=x_test, index=[i for i in range(x_test.shape[0])], columns=["comment", "target"])
    
    #print(test)
    
    #ptrain=train
    
    data_batch_size=32
    train.fillna(value='', inplace=True)
    
    train['target'] = np.where(train['target']== 0,0,1)
     
    ptrain_df = df_to_dataset(train, batch_size=data_batch_size)
    
    
    test.fillna(value='', inplace=True)
    test['target'] = np.where(test['target']== 0,0,1)
     
    ptest_df = df_to_dataset(test, batch_size=data_batch_size)
    
    
    #test['target'] = np.where(test['is_toxic']== 0,0,1)
     
    #test_df = df_to_dataset(test, batch_size=data_batch_size)
    
    val.fillna(value='', inplace=True)
    val['target'] = np.where(val['target']== 0,0,1)
     
    pval_df = df_to_dataset(val, batch_size=data_batch_size)
    
    
    #AUTOTUNE = tf.data.AUTOTUNE
    #train_dp = ptrain_df.cache().prefetch(buffer_size=AUTOTUNE)
    #val_dp = pval_df.cache().prefetch(buffer_size=AUTOTUNE)
    #test_dp = ptest_df.cache().prefetch(buffer_size=AUTOTUNE)
    
    train_dp=ptrain_df
    val_dp = pval_df
    test_dp = ptest_df
    
    ##load model
    classifier_model= build_classifier_model()

    
    ###start train
    
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metrics = tf.metrics.BinaryAccuracy()
    
    
    epochs = 10
    steps_per_epoch = tf.data.experimental.cardinality(train_dp).numpy()
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')
    
    classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    
    ##set early stopping 
    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
    
    ##training started
    print(f'Training model with {tfhub_handle_encoder}')
    history = classifier_model.fit(x=train_dp,
                               validation_data=val_dp,
                               epochs=epochs,  callbacks=[callbacks])
    
    ##testing_model
    y_tst=test['target'].values
    y_pred = tf.sigmoid(classifier_model(tf.constant(test["comment"])))
    y_pred = (y_pred >= 0.5)
    
    
    #print(y_tst, y_pred)
    ##classification metrics
    from sklearn import metrics
    print(metrics.classification_report(y_tst, y_pred))
    
    bert_precision = precision_score(y_tst, y_pred, pos_label=1)
    bert_recall = recall_score(y_tst, y_pred, pos_label=1)
    bert_f1score = f1_score(y_tst, y_pred, pos_label=1)
    bert_accuracy = accuracy_score(y_tst, y_pred)

    bert_run_accuracy.append(bert_accuracy)
    bert_run_f1score.append(bert_f1score)
    bert_run_precision.append(bert_precision)
    bert_run_recall.append(bert_recall)
    
    fold_no = fold_no+1

    
    #print(len(train))
    #print(len(val))
    #print(len(test))
    #indexs=[train_index]
    #val_indexes=[test_index]
    

Starting Fold:  1
Training model with https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Epoch 1/10
52/52 [==============================] - 18s 339ms/step - loss: 0.5793 - binary_accuracy: 0.7785 - val_loss: 0.6626 - val_binary_accuracy: 0.6522
Epoch 2/10
52/52 [==============================] - 17s 329ms/step - loss: 0.3378 - binary_accuracy: 0.8656 - val_loss: 0.4485 - val_binary_accuracy: 0.8019
Epoch 3/10
52/52 [==============================] - 17s 334ms/step - loss: 0.1725 - binary_accuracy: 0.9346 - val_loss: 0.4544 - val_binary_accuracy: 0.8068
              precision    recall  f1-score   support

           0       0.94      0.97      0.95       161
           1       0.88      0.78      0.83        46

    accuracy                           0.93       207
   macro avg       0.91      0.88      0.89       207
weighted avg       0.93      0.93      0.93       207

Starting Fold:  2
Training model with https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Epoc

52/52 [==============================] - 18s 342ms/step - loss: 0.5316 - binary_accuracy: 0.7901 - val_loss: 0.2016 - val_binary_accuracy: 1.0000
Epoch 2/10
52/52 [==============================] - 18s 342ms/step - loss: 0.3544 - binary_accuracy: 0.8433 - val_loss: 0.2098 - val_binary_accuracy: 0.9757
              precision    recall  f1-score   support

           0       0.90      0.90      0.90       156
           1       0.69      0.69      0.69        51

    accuracy                           0.85       207
   macro avg       0.79      0.79      0.79       207
weighted avg       0.85      0.85      0.85       207

